In [44]:
import numpy as np
import pandas as pd

In [45]:
file_name_Mumbai = './data/Mumbai_data.csv'
file_name_Pune = './data/Pune_data.csv'
file_name_Delhi = './data/Delhi_data.csv'

data_Mumbai = pd.read_csv(file_name_Mumbai)
data_Pune = pd.read_csv(file_name_Pune)
data_Delhi = pd.read_csv(file_name_Delhi)

data_all = [data_Mumbai, data_Pune, data_Delhi]

data = pd.concat(data_all, ignore_index=True)

In [46]:
for df in data_all:
    df.info()
    print('\n\n')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   house_type        5000 non-null   object 
 1   house_size        5000 non-null   object 
 2   location          5000 non-null   object 
 3   city              5000 non-null   object 
 4   latitude          5000 non-null   float64
 5   longitude         5000 non-null   float64
 6   price             5000 non-null   int64  
 7   currency          5000 non-null   object 
 8   numBathrooms      4986 non-null   float64
 9   numBalconies      733 non-null    float64
 10  isNegotiable      571 non-null    object 
 11  priceSqFt         0 non-null      float64
 12  verificationDate  5000 non-null   object 
 13  description       4922 non-null   object 
 14  SecurityDeposit   5000 non-null   object 
 15  Status            5000 non-null   object 
dtypes: float64(5), int64(1), object(10)
memory

In [47]:
round(data.describe(), 2)


,latitude,longitude,price,numBathrooms,numBalconies,priceSqFt
count,13910.00,13910.00,13910.00,13854.00,5291.00,0.0
mean,22.38,74.71,108497.13,2.33,1.84,NaN
std,4.66,2.18,193434.88,1.00,0.70,NaN
min,17.47,18.50,1500.00,1.00,1.00,NaN
25%,18.61,72.97,18000.00,2.00,1.00,NaN
50%,19.15,73.90,33000.00,2.00,2.00,NaN
75%,28.55,77.15,90000.00,3.00,2.00,NaN
max,31.11,91.79,3010101.00,10.00,8.00,NaN


In [48]:
data.isna().mean()


house_type          0.000000
house_size          0.000000
location            0.000000
city                0.000000
latitude            0.000000
longitude           0.000000
price               0.000000
currency            0.000000
numBathrooms        0.004026
numBalconies        0.619626
isNegotiable        0.908267
priceSqFt           1.000000
verificationDate    0.000000
description         0.059741
SecurityDeposit     0.000000
Status              0.000000
dtype: float64

In [49]:
missing_value = pd.DataFrame(data.isna().mean(), columns=['percentage'])
missing_value = missing_value[missing_value['percentage'] >= 0.5]
round(missing_value, 2)

,percentage
numBalconies,0.62
isNegotiable,0.91
priceSqFt,1.00


In [50]:
idx=missing_value.index

for i in idx:
    print(i, "\'s kinds: ", data[i].unique())

numBalconies 's kinds:  [nan  2.  4.  1.  5.  6.  3.  8.]
isNegotiable 's kinds:  [nan 'Negotiable']
priceSqFt 's kinds:  [nan]


In [51]:
data_for_EDA = data_for_model = data
data_for_EDA[idx] = data_for_EDA[idx].fillna(0)
data_for_EDA = data_for_EDA.drop('priceSqFt', axis=1)

In [52]:
data_for_EDA = data_for_EDA.dropna()
data_for_EDA.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13024 entries, 0 to 13909
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   house_type        13024 non-null  object 
 1   house_size        13024 non-null  object 
 2   location          13024 non-null  object 
 3   city              13024 non-null  object 
 4   latitude          13024 non-null  float64
 5   longitude         13024 non-null  float64
 6   price             13024 non-null  int64  
 7   currency          13024 non-null  object 
 8   numBathrooms      13024 non-null  float64
 9   numBalconies      13024 non-null  float64
 10  isNegotiable      13024 non-null  object 
 11  verificationDate  13024 non-null  object 
 12  description       13024 non-null  object 
 13  SecurityDeposit   13024 non-null  object 
 14  Status            13024 non-null  object 
dtypes: float64(4), int64(1), object(10)
memory usage: 1.6+ MB


In [53]:
data_for_EDA['house_type'].unique()

array(['2 BHK Apartment ', '3 BHK Apartment ', '1 RK Studio Apartment ',
       '1 BHK Apartment ', '4 BHK Apartment ', '1 BHK Independent Floor ',
       '3 BHK Villa ', '3 BHK Independent House ',
       '4 BHK Independent House ', '4 BHK Villa ', '6 BHK Apartment ',
       '2 BHK Independent House ', '5 BHK Apartment ', '1 BHK Villa ',
       '4 BHK Independent Floor ', '6 BHK Villa ', '5 BHK Villa ',
       '3 BHK Independent Floor ', '2 BHK Independent Floor ',
       '6 BHK penthouse ', '1 BHK Independent House ',
       '5 BHK Independent House ', '2 BHK Villa ',
       '6 BHK Independent House ', '8 BHK Independent Floor ',
       '6 BHK Independent Floor ', '5 BHK Independent Floor ',
       '7 BHK Independent Floor ', '8 BHK Independent House ',
       '10 BHK Independent House ', '7 BHK Independent House ',
       '9 BHK Independent House ', '8 BHK Villa ',
       '12 BHK Independent House '], dtype=object)

In [54]:
def normalize_house_type(house_type):
    if 'penthouse' in house_type:
        return 'Penthouse'
    elif 'Studio Apartment' in house_type:
        return 'Studio Apartment'
    elif 'Villa' in house_type:
        return 'Villa'
    elif 'Independent Floor' in house_type:
        return 'Independent Floor'
    elif 'Independent House' in house_type:
        return 'Independent House'
    else:
        return 'Other'
    
data_for_EDA['normalize_house_type'] = data_for_EDA['house_type'].apply(normalize_house_type)
print(data_for_EDA['normalize_house_type'].unique())

['Other' 'Studio Apartment' 'Independent Floor' 'Villa'
 'Independent House' 'Penthouse']


In [55]:
data_for_EDA['house_size'].unique()

array(['1,180 sq ft', '1,120 sq ft', '1,200 sq ft', ..., '6,321 sq ft',
       '333 sq ft', '1,855 sq ft'], dtype=object)

In [56]:
data_for_EDA['house_size_1000'] = (data_for_EDA['house_size'].str.replace(' sq ft', '').str.replace(',', '').astype(int) // 1000) * 1000
house_size_price_mean = data_for_EDA.groupby('house_size_1000')['price'].mean()
print(data_for_EDA['house_size_1000'].unique())

[ 1000     0  2000  3000  4000  8000  6000  5000  7000  9000 10000 14000]


In [57]:
data_for_EDA['verificationDate'].unique()


array(['Posted 2 years ago', 'Posted a month ago', 'Posted 3 years ago',
       'Posted 3 months ago', 'Posted 2 months ago', 'Posted 23 days ago',
       'Posted 18 days ago', 'Posted 11 days ago', 'Posted 4 days ago',
       'Posted a day ago', 'Posted 17 days ago', 'Posted 4 months ago',
       'Posted 12 days ago', 'Posted 8 months ago', 'Posted a year ago',
       'Posted 13 days ago', 'Posted 16 days ago', 'Posted 20 days ago',
       'Posted 19 days ago', 'Posted 22 days ago', 'Posted 9 days ago',
       'Posted 15 days ago', 'Posted 21 days ago', 'Posted 14 days ago',
       'Posted 5 days ago', 'Posted 10 days ago', 'Posted 20 hours ago',
       'Posted 8 days ago', 'Posted 2 days ago', 'Posted 24 days ago',
       'Posted 3 days ago', 'Posted 25 days ago', 'Posted 7 days ago',
       'Posted 6 days ago', 'Posted 17 hours ago', 'Posted 5 months ago',
       'Posted 19 hours ago', 'Posted 16 hours ago',
       'Posted 10 hours ago', 'Posted 21 hours ago',
       'Posted 11 hour

In [58]:
def normalize_data_type(date_str):
    if 'year' in date_str or 'years' in date_str:
        return 'year'
    elif 'month' in date_str or 'months' in date_str:
        return 'month'
    elif 'day' in date_str or 'days' in date_str:
        return 'day'
    elif 'hour' in date_str or 'hours' in date_str:
        return 'hour'
    elif 'minute' in date_str or 'minutes' in date_str:
        return 'minute'
    else:
        return 'Other'
    
data_for_EDA['date_type'] = data_for_EDA['verificationDate'].apply(normalize_data_type)
data_for_EDA['date_type'].unique()

array(['year', 'month', 'day', 'hour', 'minute'], dtype=object)